In [11]:
import pandas as pd
import numpy as np
import pickle as p

In [12]:
data_all = pd.read_csv('data_all.csv')

In [13]:
np.random.seed(41)
item_subset = list(np.random.choice(data_all['Item_ID'].unique(), size=10, replace=False))
data_subset = data_all.loc[data_all['Item_ID'].isin(item_subset)].copy()

In [14]:
item_subset

[31, 2259, 1203, 5073, 102, 5501, 3818, 4270, 3767, 197]

### `/train`

In [15]:
import requests
import json

url = 'http://localhost:5000/train/'

data = data_subset.to_json()

payload = {'data': data,
           'cv_acc': False,
           'project_id': 1
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [16]:
# Model Training Result
r.json()['result']

'Success'

In [ ]:
# CV Accuracies Returned
cv_accuracies = pd.DataFrame.from_dict(json.loads(r.json()['cv_acc']))
cv_accuracies

### `/optimize`

In [17]:
url = 'http://localhost:5000/optimize/'

payload = {'project_id': 1,
           'constraints': [],
           'population': 100,
           'max_epoch': 200
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [18]:
best_prices = r.json()['result']
best_prices

{'102': 4.24,
 '1203': 13.81,
 '197': 10.96,
 '2259': 4.03,
 '31': 14.72,
 '3767': 9.92,
 '3818': 10.82,
 '4270': 8.89,
 '5073': 4.02,
 '5501': 5.1}

In [ ]:
import requests
import json

url = 'http://localhost:5000/predict/'
payload = {'prices': best_prices,
           'project_id': 1
           }
headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }
payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

# What If Prediction
qty = r.json()['pred_q']

In [ ]:
qty

In [ ]:
# Calculate Revenue
revenue = 0
for item in qty.keys():
    revenue += (best_prices_input[item]*qty[item])
    
# Number is obscenely high due to some outliers,
# Notably 7009, likely due to lack of constraints etc.
print(revenue)

### `/predict/`

In [7]:
# Sample Actual Price 
from helper_functions import *

sales_data = data_subset

sales_data = optimize_memory(sales_data)

sales_data_wide = sales_data.set_index(
    ['Wk', 'Tier', 'Store', 'Item_ID']).unstack(level=-1).reset_index().copy()
sales_data_wide.columns = [
    ''.join(str(i) for i in col).strip()
    for col in sales_data_wide.columns.values
]
sales_data_wide = sales_data_wide.sort_values(
    ['Tier', 'Store', 'Wk'], ascending=True).reset_index(drop=True)

sales_data_wide_clean = sales_data_wide.dropna(axis=0).copy()
dataset = sales_data_wide_clean
price_columns = [
    col for col in sales_data_wide_clean.columns if col.startswith('Price')
]

a = dataset.sample(1)
a_dict = a[price_columns].iloc[0].to_dict()
a_input = {}
for key in a_dict.keys():
    a_input[key.split('_')[1]]=round(a_dict[key],2)

print('Input Prices Quantities:')
a_input

Input Prices Quantities:


{'31': 2.6,
 '102': 9.65,
 '197': 10.65,
 '1203': 3.0,
 '2259': 12.7,
 '3767': 10.25,
 '3818': 10.15,
 '4270': 7.85,
 '5073': 3.6,
 '5501': 3.5}

In [10]:
# Predict Sales Qty and Compare to Actual

import requests
import json

url = 'http://localhost:5000/predict/'

payload = {'prices': a_input, 'project_id': 1}

headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)
qty = r.json()['qty_estimates']
qty_outp = {}
for key in qty.keys():
    qty_outp[key] = round(qty[key], 0)
actual = a[[
    col for col in sales_data_wide_clean.columns if col.startswith('Qty_')
]].iloc[0].to_dict()
pd.DataFrame([qty_outp, actual], index=['Estimated', 'Actual']).transpose()

,Estimated,Actual
Qty_102,124.0,143.0
Qty_1203,176.0,243.0
Qty_197,40.0,22.0
Qty_2259,51.0,35.0
Qty_31,35.0,20.0
Qty_3767,293.0,375.0
Qty_3818,49.0,34.0
Qty_4270,29.0,12.0
Qty_5073,110.0,118.0
Qty_5501,368.0,420.0
